In [34]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [35]:
df = pd.read_csv("dataset/clean/airline-route-panel.csv")
df = df[df['origin'] != df['finaldest']]
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6522127 entries, 0 to 6527978
Data columns (total 11 columns):
 #   Column        Dtype  
---  ------        -----  
 0   airline_code  object 
 1   origin        object 
 2   finaldest     object 
 3   return        int64  
 4   year          int64  
 5   quarter       int64  
 6   airports      int64  
 7   return_sym    int64  
 8   stops         int64  
 9   passengers    float64
 10  itinfare      float64
dtypes: float64(2), int64(6), object(3)
memory usage: 597.1+ MB


In [36]:
airport_code = pd.read_csv("dataset/clean/airports.csv")
airport_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   IATA       341 non-null    object 
 1   AIRPORT    341 non-null    object 
 2   CITY       337 non-null    object 
 3   STATE      337 non-null    object 
 4   COUNTRY    341 non-null    object 
 5   LATITUDE   341 non-null    float64
 6   LONGITUDE  341 non-null    float64
dtypes: float64(2), object(5)
memory usage: 18.8+ KB


In [37]:
airport_ca = airport_code[airport_code["STATE"] == 'CA']

In [38]:
flight_ca = df[(df['origin'].isin(airport_ca['IATA']) & df['finaldest'].isin(airport_ca['IATA']))]
flight_ca.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33395 entries, 23101 to 6515761
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   airline_code  33395 non-null  object 
 1   origin        33395 non-null  object 
 2   finaldest     33395 non-null  object 
 3   return        33395 non-null  int64  
 4   year          33395 non-null  int64  
 5   quarter       33395 non-null  int64  
 6   airports      33395 non-null  int64  
 7   return_sym    33395 non-null  int64  
 8   stops         33395 non-null  int64  
 9   passengers    33395 non-null  float64
 10  itinfare      33395 non-null  float64
dtypes: float64(2), int64(6), object(3)
memory usage: 3.1+ MB


In [48]:
def season_encoding(data):
    if data == 1:
        return 'Winter'
    elif data == 2:
        return 'Spring'
    elif data == 3:
        return 'Summer'
    else:
        return 'Fall'

flight_ca['Season'] = flight_ca['quarter'].apply(season_encoding)

In [40]:
#merge lat and long for origin
flight_ca = flight_ca.merge(airport_ca, how='left', left_on = 'origin', right_on = 'IATA')
flight_ca.rename(columns = {'latitude': 'origin_latitude', 'longitude': 'origin_longitude'}, inplace = True)

#merge for dest
flight_ca = flight_ca.merge(airport_ca, how='left', left_on = 'finaldest', right_on = 'IATA')
flight_ca.rename(columns = {'latitude': 'origin_latitude', 'longitude': 'origin_longitude'}, inplace = True)

In [41]:
airline = pd.read_csv('airlines.csv')
airline.rename(columns={'Description' : 'Airline'}, inplace = True)
airline.head()

,Code,Airline
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.


In [42]:
flight_ca = flight_ca.merge(airline, how = 'left', left_on='airline_code', right_on='Code')

In [43]:
flight_ca = pd.get_dummies(flight_ca, columns = ['Season'], drop_first = True)

In [49]:
flight_ca.head()

,airline_code,origin,finaldest,return,year,quarter,airports,return_sym,stops,passengers,...,STATE_y,COUNTRY_y,LATITUDE_y,LONGITUDE_y,Code,Airline,Season_Spring,Season_Summer,Season_Winter,Season
0,AA,BUR,LAX,0,2010,2,4,0,2,42.0,...,CA,USA,33.942536,-118.408074,AA,American Airlines Inc.,True,False,False,Spring
1,AA,BUR,ONT,0,2010,2,3,1,1,1.0,...,CA,USA,34.056000,-117.601194,AA,American Airlines Inc.,True,False,False,Spring
2,AA,BUR,SAN,0,2010,2,3,1,1,1.0,...,CA,USA,32.733556,-117.189657,AA,American Airlines Inc.,True,False,False,Spring
3,AA,BUR,SFO,0,2010,2,4,0,2,2.0,...,CA,USA,37.619002,-122.374843,AA,American Airlines Inc.,True,False,False,Spring
4,AA,BUR,SJC,0,2010,2,3,1,1,1.0,...,CA,USA,37.361862,-121.929009,AA,American Airlines Inc.,True,False,False,Spring


In [45]:
X = flight_ca[['Season_Spring', 'Season_Summer', 'Season_Winter']].astype(int)
Y = flight_ca[['itinfare']]

In [46]:
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
coef = pd.DataFrame(model.params).reset_index()
coef.to_csv('seasonality.csv')

In [51]:
flight_ca.to_csv('flight_ca.csv', index = False)